#### US Financial Health

In this project, we’ll be importing various types of financial data to try and determine the financial health and volatility of the US between 1999 and 2019.

We’ll use the techniques we’ve learned for importing financial data, to import stock and commodity pricing data from csv files and the FRED API. Then grab GDP and goods and services export data from the World Bank API.

Finally, we’ll find the log returns of the imported data, and use that to determine the volatility of the data over the 20 year period.


In [1]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import pandas_datareader.wb as wb 
import numpy as np

In [2]:
gold_prices = pd.read_csv('gold_prices.csv')
crude_oil_prices = pd.read_csv('crude_oil_prices.csv')

In [3]:
print(gold_prices.head())

         Date  Gold_Price
0  2019-08-30     1528.40
1  2019-08-29     1540.20
2  2019-08-28     1537.15
3  2019-08-27     1532.95
4  2019-08-26     1503.80


In [4]:
print(crude_oil_prices.head())

           Date  Crude_Oil_Price
0  Sep 11, 2018            69.25
1  Sep 10, 2018            67.54
2  Sep 07, 2018            67.75
3  Sep 06, 2018            67.77
4  Sep 05, 2018            68.72


In [5]:
start = datetime(1999,1,1)
end = datetime(2019,1,1)
nasdaq_data = web.DataReader('NASDAQ100', 'fred', start, end)
print(nasdaq_data.tail())

            NASDAQ100
DATE                 
2018-12-26   6262.766
2018-12-27   6288.301
2018-12-28   6285.266
2018-12-31   6329.965
2019-01-01        NaN


In [6]:
gdp_data = wb.download(indicator='NY.GDP.MKTP.CD', country=['US'], start=start, end=end)
print(gdp_data)

                    NY.GDP.MKTP.CD
country       year                
United States 2019             NaN
              2018    2.049410e+13
              2017    1.948539e+13
              2016    1.870719e+13
              2015    1.821930e+13
              2014    1.752175e+13
              2013    1.678485e+13
              2012    1.619701e+13
              2011    1.554258e+13
              2010    1.499205e+13
              2009    1.444893e+13
              2008    1.471284e+13
              2007    1.445186e+13
              2006    1.381461e+13
              2005    1.303664e+13
              2004    1.221373e+13
              2003    1.145824e+13
              2002    1.093642e+13
              2001    1.058182e+13
              2000    1.025235e+13
              1999    9.630664e+12


In [7]:
export_data = wb.download(indicator='NE.EXP.GNFS.CN', country=['US'], start=start, end=end)
print(export_data)

                    NE.EXP.GNFS.CN
country       year                
United States 2019             NaN
              2018             NaN
              2017    2.350175e+12
              2016    2.217576e+12
              2015    2.265047e+12
              2014    2.371027e+12
              2013    2.273428e+12
              2012    2.191280e+12
              2011    2.102995e+12
              2010    1.846280e+12
              2009    1.581996e+12
              2008    1.837055e+12
              2007    1.660853e+12
              2006    1.472613e+12
              2005    1.305225e+12
              2004    1.177631e+12
              2003    1.036177e+12
              2002    9.987410e+11
              2001    1.024636e+12
              2000    1.096255e+12
              1999    9.927780e+11


In [8]:
def log_return(prices):
    return np.log(prices/prices.shift(1))

gold_returns = log_return(gold_prices['Gold_Price'])
crude_oil_returns = log_return(crude_oil_prices['Crude_Oil_Price'])
nasdaq_returns = log_return(nasdaq_data['NASDAQ100'])
gdp_change = log_return(gdp_data['NY.GDP.MKTP.CD'])
export_change = log_return(export_data['NE.EXP.GNFS.CN'])

In [9]:
print(gold_returns.var())
print(crude_oil_returns.var())
print(nasdaq_returns.var())

print(gdp_change.var())
print(export_change.var())

0.00011375928671558533
0.0005563207795629894
0.0003178379833057235
0.0003576341221890429
0.006775724487950142



The S&P 500, a collection of 500 large companies listed on stock exchanges in the United States, has the smallest variance, and thus is the least volatile. Given that the S&P 500 index is a weighted measurement of many stocks across a variety of industries, it is seen as a safer, diversified investment.

Gold, notorious for being a stable investment has the second smallest variance.

Crude oil is the most volatile, which makes sense as gas prices are often unpredictable, especially in the last 20 years.

The stocks are interesting. The NASDAQ 100 is more volatile than the S&P 500, which, when you think about it makes sense, as the S&P 500 is far more diversified and tracks more of the market.

Then finally we have GDP and exports.

Exports are very volatile, which could have to do with industries moving overseas in the last 20 years, and global competition for the production of goods.

GDP is actually fairly similar to the NASDAQ 100 in terms of volatility, which is perhaps an interesting correlation.
